# Inicialización de los pesos de una red

Veamos un ejemplo (es el que saldrá en el examen)

### Fashion-MNIST con inicialización RandomNormal
- MLP inicial: MLP con una capa oculta de 800 RELUs, batch size 16, 20 épocas; 88.0% en test
- Mejor arquitectura: una capa oculta de 800 RELUs, 89.0% en val, 88.3% en test ( 88.0% modelo val)
- Learning rate y batch size: ajustados a 0.00015 y 256; 89.6% en val, 89.8% en test (89.1% modelo val)
- ReduceLROnPlateau: factor 0.32 y paciencia 5; en val, 89.6% en test (89.5% modelo val)

Primero hacemos la inicialización de la semilla, la librería, la partición del datasey y la lectura del set.

### Explicación del Código

Este script de Python es una preparación previa para entrenar una red neuronal en el conjunto de datos Fashion MNIST utilizando Keras y Keras Tuner

1. **Importación de Librerías**:
   - `numpy` y `matplotlib.pyplot`: Librerías comúnmente usadas para operaciones numéricas y visualización.
   - `os`: Se utiliza para establecer variables de entorno y suprimir los mensajes extensos de TensorFlow.
   - `keras` y `keras_tuner`: Para construir, entrenar y ajustar modelos de redes neuronales.

2. **Configuración de Semilla Aleatoria**:
   - `keras.utils.set_random_seed(23)`: Establece una semilla aleatoria para garantizar la reproducibilidad de los resultados.

3. **Definición de Dimensiones de Entrada y Clases**:
   - `input_dim = 784`: Representa la dimensión de entrada (28x28 píxeles convertidos en un vector de 784 valores).
   - `num_classes = 10`: Representa las 10 clases en las que se clasifica Fashion MNIST.

4. **Carga de Datos**:
   - `keras.datasets.fashion_mnist.load_data()`: Carga el conjunto de datos Fashion MNIST, dividido en datos de entrenamiento y prueba.

5. **Normalización y Redimensionamiento**:
   - `x_train_val` y `x_test`: Se redimensionan los datos de imágenes de 28x28 a vectores de 784 y se normalizan dividiendo entre 255.0 para escalar los valores entre 0 y 1.

6. **Codificación One-Hot de Etiquetas**:
   - `keras.utils.to_categorical`: Convierte las etiquetas en formato one-hot, necesario para el entrenamiento de la red neuronal.

7. **División de Datos de Validación**:
   - Se dividen los datos de entrenamiento en dos conjuntos:
     - `x_train` y `y_train`: Conjunto de entrenamiento (todas las muestras excepto las últimas 10,000).
     - `x_val` y `y_val`: Conjunto de validación (las últimas 10,000 muestras).

### Resultado:
Este código deja los datos listos para entrenar un modelo de red neuronal, con conjuntos de datos normalizados y divididos en entrenamiento, validación y prueba.


In [1]:
import numpy as np; import matplotlib.pyplot as plt
import os; os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'
import keras; import keras_tuner

keras.utils.set_random_seed(23); input_dim = 784; num_classes = 10

(x_train_val, y_train_val), (x_test, y_test) = keras.datasets.fashion_mnist.load_data()
x_train_val = x_train_val.reshape(-1, input_dim).astype("float32") / 255.0
x_test = x_test.reshape(-1, input_dim).astype("float32") / 255.0

y_train_val = keras.utils.to_categorical(y_train_val, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)
x_train = x_train_val[:-10000]; x_val = x_train_val[-10000:]
y_train = y_train_val[:-10000]; y_val = y_train_val[-10000:]

2024-11-28 18:15:12.053716: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-11-28 18:15:12.187709: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-11-28 18:15:12.231693: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1452] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


### Alternativas de hipermodelo

- Exploramos inicialización RandomNormal con desviación estándar entre 0.01 y 0.10

In [3]:
class MyHyperModel(keras_tuner.HyperModel):
  def build(self, hp):
    M = keras.Sequential()
    M.add(keras.Input(shape=(784,)))

    # La desviación estándar viene dada entre min_value y max_value
    stddev = hp.Float("stddev", min_value=0.01, max_value=0.10)
    kernel_initializer = keras.initializers.RandomNormal(stddev=stddev)

    M.add(keras.layers.Dense(units=800, activation='relu', kernel_initializer=kernel_initializer))
    M.add(keras.layers.Dense(10, activation='softmax'))

    opt = keras.optimizers.Adam(learning_rate=0.00015)

    M.compile(loss="categorical_crossentropy", optimizer=opt, metrics=["accuracy"])
    return M

  def fit(self, hp, M, x, y, xy_val, **kwargs):
    factor = 0.32; patience = 5
    
    reduce_cb = keras.callbacks.ReduceLROnPlateau(monitor='val_accuracy', factor=factor, patience=patience, min_delta=1e-4, min_lr=1e-5)
    
    early_cb = keras.callbacks.EarlyStopping(monitor='val_accuracy', patience=2*patience, min_delta=1e-5)
    
    kwargs['callbacks'].extend([reduce_cb, early_cb])
    return M.fit(x, y, batch_size=256, epochs=100, validation_data=xy_val, **kwargs)

- Learning rate inicial y final con 10 decay_steps y power por omisión

In [ ]:
class MyHyperModel(keras_tuner.HyperModel):
  def build(self, hp):
    M = keras.Sequential()
    M.add(keras.Input(shape=(784,)))
    M.add(keras.layers.Dense(units=800, activation='relu'))
    M.add(keras.layers.Dense(10, activation='softmax'))
    
    initial_learning_rate = hp.Float("initial_learning rate", min_value=0.2983, max_value=0.3200)
    end_learning_rate = hp.Float("end_learning rate", min_value=0.2800, max_value=0.2983)
    
    decay_steps = 10
    
    lr_schedule = keras.optimizers.schedules.PolynomialDecay(initial_learning_rate,
    decay_steps, end_learning_rate)
    
    opt = keras.optimizers.SGD(learning_rate=lr_schedule)
    M.compile(loss="categorical_crossentropy", optimizer=opt, metrics=["accuracy"])
    return M

  def fit(self, hp, M, x, y, xy_val, **kwargs):
    patience = 10
    
    early_cb = keras.callbacks.EarlyStopping(monitor='val_accuracy', patience=2*patience, min_delta=0.0)
    
    kwargs['callbacks'].append(early_cb)
    return M.fit(x, y, batch_size=256, epochs=100, validation_data=xy_val, **kwargs)

- Exploramos learning rate, momentum y Nesterov

In [ ]:
class MyHyperModel(keras_tuner.HyperModel):
  def build(self, hp):
    M = keras.Sequential()
    M.add(keras.Input(shape=(784,)))
    M.add(keras.layers.Dense(units=800, activation='relu'))
    M.add(keras.layers.Dense(10, activation='softmax'))
    
    learning_rate = hp.Float("learning rate", min_value=0.25, max_value=0.35)
    momentum = hp.Float("momentum", min_value=0.1, max_value=0.2)
    nesterov = hp.Boolean("nesterov")
    
    opt = keras.optimizers.SGD(learning_rate=learning_rate, momentum=momentum, nesterov=nesterov)
    
    M.compile(loss="categorical_crossentropy", optimizer=opt, metrics=["accuracy"])
    return M

  def fit(self, hp, M, x, y, xy_val, **kwargs):
    factor = 0.3787; patience = 10
    reduce_cb = keras.callbacks.ReduceLROnPlateau(
    monitor='val_accuracy', factor=factor, patience=patience, min_delta=1e-5, min_lr=0.0)
    
    early_cb = keras.callbacks.EarlyStopping(monitor='val_accuracy', patience=2*patience, min_delta=1e-5)
    
    kwargs['callbacks'].extend([reduce_cb, early_cb])
    return M.fit(x, y, batch_size=256, epochs=100, validation_data=xy_val, **kwargs)

Ahora hacemos una evaluación en test del mejor modelo en validación

In [4]:
tuner = keras_tuner.BayesianOptimization(
MyHyperModel(), objective="val_accuracy", max_trials=10, executions_per_trial=1,
overwrite=True, directory="/tmp", project_name="Fashion-MNIST")

In [5]:
tuner.search(x_train, y_train, (x_val, y_val))

Trial 10 Complete [00h 01m 22s]
val_accuracy: 0.8970999717712402

Best val_accuracy So Far: 0.900600016117096
Total elapsed time: 00h 17m 05s


In [6]:
tuner.results_summary(num_trials=1)

Results summary
Results in /tmp/Fashion-MNIST
Showing 1 best trials
Objective(name="val_accuracy", direction="max")

Trial 02 summary
Hyperparameters:
stddev: 0.07271143839834814
Score: 0.900600016117096


In [7]:
best = tuner.get_best_models(num_models=1)[0]
score = best.evaluate(x_test, y_test, verbose=0)
print(f'Loss: {score[0]:.4}\nPrecisión: {score[1]:.2%}')

/home/yassin/anaconda3/envs/per/lib/python3.11/site-packages/keras/src/saving/saving_lib.py:719: UserWarning: Skipping variable loading for optimizer 'adam', because it has 2 variables whereas the saved optimizer has 10 variables. 
  saveable.load_own_variables(weights_store.get(inner_path))


Loss: 0.3103
Precisión: 89.55%
